# Non-negative matrix factorization using Autograd

In a [previous post](./nnmf-tensorflow.html), we had seen how to perfom non-negative matrix factorization (NNMF) using Tensorflow. In this post, we will look at performing NNMF using [Autograd](https://github.com/HIPS/autograd). Like Tensorflow, Autograd allows automatic gradient calculation.

### Customary imports

In [1]:
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Creating the matrix to be decomposed

In [2]:
A = np.array([[3, 4, 5, 2],
                   [4, 4, 3, 3],
                   [5, 5, 4, 3]], dtype=np.float32).T

### Masking one entry

In [3]:
A[0, 0] = np.NAN

In [4]:
A

array([[ nan,   4.,   5.],
       [  4.,   4.,   5.],
       [  5.,   3.,   4.],
       [  2.,   3.,   3.]], dtype=float32)

### Defining the cost function

In [5]:
def cost(W, H):
    pred = np.dot(W, H)
    mask = ~np.isnan(A)
    return np.sqrt(((pred - A)[mask].flatten() ** 2).mean(axis=None))

### Decomposition params

In [43]:
rank = 2
learning_rate=0.01
n_steps = 10000

### Gradient of cost wrt params W and H

In [44]:
from autograd import grad, multigrad
grad_cost= multigrad(cost, argnums=[0,1])

### Main gradient descent routine

In [45]:
shape = A.shape
H =  np.abs(np.random.randn(rank, shape[1]))
W =  np.abs(np.random.randn(shape[0], rank))

In [46]:
H

array([[ 0.72485429,  0.74836855,  1.58077507],
       [ 0.14492612,  2.0299489 ,  1.59780125]])

In [47]:
W

array([[ 1.83422364,  0.83966732],
       [ 0.57747077,  0.35783761],
       [ 0.49182969,  0.6287203 ],
       [ 0.45418243,  0.25230677]])

In [48]:
cost(W, H)

2.5692765910946553

In [49]:
del_W, del_H = grad_cost(W, H)

In [50]:
del_W

array([[-0.06688302, -0.10918776],
       [-0.36239114, -0.42093278],
       [-0.27671163, -0.2461087 ],
       [-0.20388266, -0.26888192]])

In [51]:
del_H*del_H

array([[ 0.03154308,  0.03099769,  0.03605208],
       [ 0.02577669,  0.01270763,  0.01773453]])

In [52]:
np.square(del_H)

array([[ 0.03154308,  0.03099769,  0.03605208],
       [ 0.02577669,  0.01270763,  0.01773453]])

In [53]:
gt_w = np.zeros_like(W)
gt_h = np.zeros_like(H)
eps = 1e-8
print "Iteration, Cost"
for i in range(n_steps):
    
    if i%1000==0:
        print "*"*20
        print i,",", cost(W, H)
    
    del_W, del_H = grad_cost(W, H)
    gt_w+= np.square(del_W)
    gt_h+= np.square(del_H)
    
    mod_learning_rate_W = np.divide(learning_rate, np.sqrt(gt_w+eps))
    mod_learning_rate_H = np.divide(learning_rate, np.sqrt(gt_h+eps))
    W =  W-del_W*mod_learning_rate_W
    H =  H-del_H*mod_learning_rate_H
    
    # Ensuring that W, H remain non-negative. This is also called projected gradient descent
    W[W<0] = 0
    H[H<0] = 0

Iteration, Cost
********************
0 , 2.56927659109
********************
1000 , 0.951717169579
********************
2000 , 0.581951119827
********************
3000 , 0.398336552191
********************
4000 , 0.289701848232
********************
5000 , 0.19633884565
********************
6000 , 0.121571316243
********************
7000 , 0.0906007414958
********************
8000 , 0.0868142233085
********************
9000 , 0.0865688672293


In [54]:
pd.DataFrame(W)

,0,1
0,1.729911,1.104788
1,1.191779,1.333739
2,2.018141,0.570662
3,0.355614,1.083090


In [55]:
pd.DataFrame(H)

,0,1,2
0,2.157829,0.783644,1.312347
1,1.101042,2.393814,2.470611


In [56]:
pred = np.dot(W, H)
pred_df = pd.DataFrame(pred).round()
pred_df

,0,1,2
0,5.0,4.0,5.0
1,4.0,4.0,5.0
2,5.0,3.0,4.0
3,2.0,3.0,3.0


In [57]:
pd.DataFrame(A)

,0,1,2
0,NaN,4.0,5.0
1,4.0,4.0,5.0
2,5.0,3.0,4.0
3,2.0,3.0,3.0


In [58]:
gt_w

array([[  2.44242307e-01,   5.63349664e-01],
       [  2.99507989e+02,   2.51168511e+02],
       [  3.31684975e+02,   2.53301910e+01],
       [  3.29582472e+01,   3.93531649e+01]])

In [59]:
gt_h

array([[ 527.75972021,   13.77089759,   27.28864048],
       [ 303.27252759,    5.94133343,   23.39096651]])

In [60]:
mod_learning_rate_H

array([[ 0.00043529,  0.00269475,  0.0019143 ],
       [ 0.00057423,  0.00410259,  0.00206764]])

In [61]:
mod_learning_rate_W

array([[ 0.02023436,  0.01332327],
       [ 0.00057782,  0.00063098],
       [ 0.00054908,  0.00198692],
       [ 0.00174188,  0.00159408]])